5. ПОСТРОЕНИЕ РЕКОМЕНДАЦИЙ ПО ПОСЛЕДОВАТЕЛЬНОСТЯМ ДЕЙСТВИЙ

Лекция:
https://www.youtube.com/watch?v=4HMnWES84qg
Ноутбук с практикой:
https://disk.yandex.ru/d/B8rzuN2OExqi6A
Конспект notion:
https://www.notion.so/5-2abf000b96fa4571ac7ef4e081080d3e
Конспект pdf:
https://disk.yandex.ru/i/BGK4wQV0xr-zlw
Презентация:
https://disk.yandex.ru/i/X0YWKHQhDsg0-A


Ноутбук для домашней работы
https://disk.yandex.ru/d/-8vGs19jdo6iWg
Датасет для домашней работы train.parquet
https://disk.yandex.ru/d/I-0aOHn-0pjCKA
Ноутбук с готовым решением от автора:
https://disk.yandex.ru/d/3CSF5LNE5hdccA

In [1]:
# расскоментируйте код ниже, чтобы установить все зависимости
# !pip install -q \
#     pyarrow==12.0.1 \
#     polars==0.18.6 \
#     pandas==2.0.3 \
#     optuna==3.3.0 \
#     tqdm==4.65.0 \
#     numpy==1.24.3 \
#     redis==4.6.0 \
#     gensim==4.3.2

In [1]:
# раскоментируйте код ниже, чтобы скачать данные
# !wget -q https://files.grouplens.org/datasets/movielens/ml-100k.zip
# !unzip -q ml-100k.zip

In [2]:
import uuid
import redis
import optuna
import random

import polars as pl
import pandas as pd
import numpy as np
from tqdm import tqdm
from gensim.models import Word2Vec

from typing import List, Any

/home/jupyter-pavel.lyubovin/.conda/envs/recsys/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ratings = pd.read_csv(
    'ml-100k/u.data', sep='\t',
    names=['user_id', 'item_id', 'rating', 'timestamp']
)
ratings = pl.from_pandas(ratings).filter(pl.col('rating') >= 4)
ratings

user_id,item_id,rating,timestamp
i64,i64,i64,i64
298,474,4,884182806
253,465,5,891628467
286,1014,5,879781125
200,222,5,876042340
122,387,5,879270459
291,1042,4,874834944
119,392,4,886176814
167,486,4,892738452
299,144,4,877881320


In [4]:
grouped_df = (
    ratings
    .groupby('user_id')
    .agg([
        pl.col('item_id').apply(lambda x: x[:-3]).alias('train_ids'),
        pl.col('rating').apply(lambda x: x[:-3]).alias('train_ratings'),
        pl.col('item_id').apply(lambda x: x[-3:]).alias('test_ids'),
        pl.col('rating').apply(lambda x: x[-3:]).alias('test_ratings'),
    ])
)
grouped_df

user_id,train_ids,train_ratings,test_ids,test_ratings
i64,list[i64],list[i64],list[i64],list[i64]
480,"[190, 183, … 56]","[5, 4, … 4]","[302, 169, 98]","[4, 5, 4]"
608,"[1172, 490, … 187]","[5, 4, … 4]","[216, 1281, 93]","[5, 4, 4]"
384,"[272, 355, … 327]","[5, 4, … 4]","[879, 258, 286]","[4, 4, 4]"
96,"[156, 87, … 514]","[4, 4, … 4]","[196, 234, 423]","[4, 4, 5]"
768,"[173, 282, … 272]","[5, 4, … 5]","[100, 222, 301]","[5, 4, 5]"
192,"[1061, 1160, … 100]","[4, 4, … 5]","[289, 9, 287]","[4, 5, 4]"
416,"[125, 724, … 291]","[5, 4, … 4]","[1594, 79, 153]","[5, 5, 4]"
448,"[360, 262, … 258]","[4, 4, … 4]","[1062, 1022, 270]","[5, 5, 5]"
160,"[234, 174, … 250]","[5, 5, … 4]","[488, 952, 127]","[5, 4, 5]"


In [5]:
TOP_K = 10


def user_intersection(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> int:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: number of items in intersection of y_rel and y_rec (truncated to top-K)
    """
    return len(set(y_rec[:k]).intersection(set(y_rel)))


def user_hitrate(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> int:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: 1 if top-k recommendations contains at lease one relevant item
    """
    return int(user_intersection(y_rel, y_rec, k) > 0)


def user_precision(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> float:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: percentage of relevant items through recommendations
    """
    return user_intersection(y_rel, y_rec, k) / k


def user_ap(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> float:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: average precision metric for user recommendations
    """
    return np.sum([
        user_precision(y_rel, y_rec, idx + 1)
        for idx, item in enumerate(y_rec[:k]) if item in y_rel
    ]) / k

## Применим алгоритм w2v

Внутри gensim написан эффективный метод `predict_output_word`, позволяющий получать следующие возможные токены (в нашем случае объекты для рекомендации), перед фильтрацией просмотренного попросим его сгенерировать на `len(train_ids)` токенов больше (в худшем случае, нам придется все отфильтровать)

In [6]:
def evaluate_model(model):
    ap_list = []
    hitrate_list = []
    for train_ids, y_rel in grouped_df.select('train_ids', 'test_ids').rows():
        model_preds = model.predict_output_word(
            train_ids, topn=(TOP_K + len(train_ids))
        )
        if model_preds is None:
            hitrate_list.append(0)
            continue

        y_rec = [pred[0] for pred in model_preds if pred[0] not in train_ids]
        ap_list.append(user_ap(y_rel, y_rec))
        hitrate_list.append(user_hitrate(y_rel, y_rec))
    return np.mean(ap_list), np.mean(hitrate_list)

# обучим w2v с параметрами по умолчанию
model = Word2Vec(grouped_df['train_ids'].to_list())
mean_ap, mean_hitrate = evaluate_model(model)
print(f'MAP@{TOP_K} = {mean_ap:.4f} Hitrate@{TOP_K} = {mean_hitrate:.4f}')

MAP@10 = 0.0038 Hitrate@10 = 0.1348


## Подберем самые оптимальные гиперпараметры с помощью optuna

Дла алгоритма [W2V](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec) рассмотрим следующие параметры:

- `sg` – 1, если использовать skip-gram, иначе cbow
- `window` – размер окна для обучения алгоритма w2v
- `ns_exponent` – степень популярности объектов, которая будет использована для negative sampling
- `negative` – количество негативных примеров для сэмплирования
- `min_count` – минимальное число взаимодействий, нужное для 
- `vector_size` – размерность эмбеддингов

In [7]:
SEED = 42

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)

def objective(trial):
    sg = trial.suggest_categorical('sg', [0, 1])
    window = trial.suggest_int('window', 1, 10)
    ns_exponent = trial.suggest_float('ns_exponent', -3, 3)
    negative = trial.suggest_int('negative', 3, 20)
    min_count = trial.suggest_int('min_count', 0, 20)
    vector_size = trial.suggest_categorical('vector_size', [16, 32, 64, 128])
    
    print({
        'sg': sg,
        'window_len': window,
        'ns_exponent': ns_exponent,
        'negative': negative,
        'min_count': min_count,
        'vector_size': vector_size,
    })
    
    set_seed(SEED)
    model = Word2Vec(
        grouped_df['train_ids'].to_list(),
        window=window,
        sg=sg,
        hs=0,
        min_count=min_count,
        vector_size=vector_size,
        negative=negative,
        ns_exponent=ns_exponent,
        seed=SEED,
        epochs=10,
    )
    
    mean_ap, mean_hitrate = evaluate_model(model)
    print(f'MAP@{TOP_K} = {mean_ap:.4f} Hitrate@{TOP_K} = {mean_hitrate:.4f}')
    return mean_ap
    
    
study = optuna.create_study(directions=('maximize',))
study.optimize(objective, n_trials=100)

study.best_params

[I 2024-04-04 14:45:38,349] A new study created in memory with name: no-name-3bac1fd7-d3b6-4a5c-9be2-89bb97235b79


{'sg': 0, 'window_len': 10, 'ns_exponent': -0.3960956499056234, 'negative': 15, 'min_count': 12, 'vector_size': 32}


[I 2024-04-04 14:45:39,757] Trial 0 finished with value: 0.021630656872678147 and parameters: {'sg': 0, 'window': 10, 'ns_exponent': -0.3960956499056234, 'negative': 15, 'min_count': 12, 'vector_size': 32}. Best is trial 0 with value: 0.021630656872678147.


MAP@10 = 0.0216 Hitrate@10 = 0.4363
{'sg': 0, 'window_len': 10, 'ns_exponent': 0.35353863861027346, 'negative': 20, 'min_count': 9, 'vector_size': 16}


[I 2024-04-04 14:45:41,973] Trial 1 finished with value: 0.018640661938534284 and parameters: {'sg': 0, 'window': 10, 'ns_exponent': 0.35353863861027346, 'negative': 20, 'min_count': 9, 'vector_size': 16}. Best is trial 0 with value: 0.021630656872678147.


...

[I 2024-04-04 14:49:37,902] Trial 98 finished with value: 0.021803275920297193 and parameters: {'sg': 1, 'window': 2, 'ns_exponent': -0.3813959000484169, 'negative': 20, 'min_count': 18, 'vector_size': 64}. Best is trial 78 with value: 0.03334101654846335.


MAP@10 = 0.0218 Hitrate@10 = 0.4055
{'sg': 1, 'window_len': 8, 'ns_exponent': -0.012988161118688992, 'negative': 16, 'min_count': 20, 'vector_size': 64}


[I 2024-04-04 14:49:42,674] Trial 99 finished with value: 0.034305091185410336 and parameters: {'sg': 1, 'window': 8, 'ns_exponent': -0.012988161118688992, 'negative': 16, 'min_count': 20, 'vector_size': 64}. Best is trial 99 with value: 0.034305091185410336.


MAP@10 = 0.0343 Hitrate@10 = 0.5414


{'sg': 1,
 'window': 8,
 'ns_exponent': -0.012988161118688992,
 'negative': 16,
 'min_count': 20,
 'vector_size': 64}

In [8]:
set_seed(SEED)
model = Word2Vec(
    grouped_df['train_ids'].to_list(),
    **study.best_params,
    hs=0,
    seed=SEED,
    epochs=50
)

hitrate_list = []
for train_ids, y_rel in grouped_df.select('train_ids', 'test_ids').rows():
    model_preds = model.predict_output_word(train_ids, topn=(TOP_K + len(train_ids)))
    if model_preds is None:
        hitrate_list.append(0)
        continue
        
    y_rec = [pred[0] for pred in model_preds if pred[0] not in train_ids]
    hitrate_list.append(user_hitrate(y_rel, y_rec))

mean_ap, mean_hitrate = evaluate_model(model)
print(f'MAP@{TOP_K} = {mean_ap:.4f} Hitrate@{TOP_K} = {mean_hitrate:.4f}')

MAP@10 = 0.0370 Hitrate@10 = 0.5563


## Сохраним рекомендации в redis

В реальных системах в качестве идентификаторов скорее всего будет использоваться uuid, тогда как при обучении удобно использовать представление в виде целых чисел.

Одним из вариантов работы с идентификаторами является создание словарей `user_ids_mapping` и `user_ids_inverse_mapping`, где первый делает преобразование _uuid -> int_, а второй _int -> uuid_.

Давайте просимулируем реальную рекомендательную систему и загрузим рекомендации в **redis**.

In [9]:
user_ids_inverse_mapping = {k: uuid.uuid4() for k in ratings['user_id'].unique()}
item_ids_inverse_mapping = {k: uuid.uuid4() for k in ratings['item_id'].unique()}

In [13]:
# если redis запущен в том же окружении, то можно использовать localhost
# иначе, измените host на ip-адресс сервера с запущенным redis
r = redis.Redis(host='localhost', db=0)
used_memory_before = r.info('memory')['used_memory']

In [14]:
TOP_K = 100  # сохраним топ-100 рекомендаций

for user_id, train_ids in grouped_df.select('user_id', 'train_ids').rows():
    model_preds = model.predict_output_word(train_ids, topn=(TOP_K + len(train_ids)))
    if model_preds is None:
        continue
        
    y_rec = [pred[0] for pred in model_preds if pred[0] not in train_ids]
    r.rpush(
        str(user_ids_inverse_mapping[user_id]),
        *[str(item_ids_inverse_mapping[item_id]) for item_id in y_rec]
    )

In [15]:
# прочитаем сохраненные рекомендации по ключу user_id
r.lrange(str(user_ids_inverse_mapping[1]), 0, TOP_K)

[b'fe99625b-9d3f-4cd6-a0cf-f718646a7318',
 b'238f70b5-09be-4220-b86d-8e8bd7d89099',
 b'7fd55bd5-f77c-4f45-b3d4-e008bb193c1a',
 b'58f7de35-0fbc-4b6f-900e-69865d33030c',
 b'cb562984-b3d8-4523-87b6-cb8eaec8e083',
 b'2b86535d-0175-4345-a7b9-a730ef70e80e',
 b'b3711e98-b96d-431e-98bf-1c4f2b38021f',
 b'ac374fea-7aad-46da-af7b-1435c769b35f',
 b'eebc547e-bd11-46db-9a17-5d5f224bbeac',
 b'5461eb58-a836-48b2-a691-c7773a9a1d01',
 b'4c48de2e-a3e2-4aa2-92d6-3091f6cfa66f',
 b'17960fc9-58c5-474b-849b-1d3343ae0878',
 b'37ceac17-2f2c-494f-b442-f0469375acce',
 b'6363813d-e1dc-4ac2-9062-da0e87bcfb6d',
 b'a68c2ac0-fa4c-4182-8af2-a7435098e658',
 b'57243f1d-1c6a-4a19-adc4-97e57e588389',
 b'aa7b5cdc-ea58-4354-b8de-60dbe6cc19c3',
 b'ec9261fc-402b-4f49-9e8c-89443084c85c',
 b'4b533ec7-02d4-4e52-a56d-90f15018e685',
 b'0c9e9bee-0b26-4df6-9160-8fd53959ec26',
 b'37685ac6-ed01-4682-a484-6c19f380ea98',
 b'cbf5ee26-b81f-44ab-bf28-29a18d4f1078',
 b'b7867d5c-d6b8-4e54-b1e9-d34ab0f51e78',
 b'4c5db45a-a588-422a-ae44-5fc7b57

In [16]:
r.info('memory')['used_memory'] - used_memory_before

4415048

Потратили около 5Мб на сохранение рекомендаций

In [17]:
r = redis.Redis(db=1)
used_memory_before = r.info('memory')['used_memory']

In [18]:
TOP_K = 100  # сохраним топ-100 рекомендаций

for user_id, train_ids in grouped_df.select('user_id', 'train_ids').rows():
    model_preds = model.predict_output_word(train_ids, topn=(TOP_K + len(train_ids)))
    if model_preds is None:
        continue
        
    y_rec = [pred[0] for pred in model_preds if pred[0] not in train_ids]
    r.rpush(
        user_ids_inverse_mapping[user_id].bytes,
        *[item_ids_inverse_mapping[item_id].bytes for item_id in y_rec]
    )

In [19]:
# прочитаем сохраненные рекомендации по ключу user_id
# для этого сначала преобразуем user_uuid в байты, а затем байты в item_uuid
[
    uuid.UUID(bytes=item_id_bytes, version=4)
    for item_id_bytes in r.lrange(user_ids_inverse_mapping[1].bytes, 0, TOP_K)
]

[UUID('fe99625b-9d3f-4cd6-a0cf-f718646a7318'),
 UUID('238f70b5-09be-4220-b86d-8e8bd7d89099'),
 UUID('7fd55bd5-f77c-4f45-b3d4-e008bb193c1a'),
 UUID('58f7de35-0fbc-4b6f-900e-69865d33030c'),
 UUID('cb562984-b3d8-4523-87b6-cb8eaec8e083'),
 UUID('2b86535d-0175-4345-a7b9-a730ef70e80e'),
 UUID('b3711e98-b96d-431e-98bf-1c4f2b38021f'),
 UUID('ac374fea-7aad-46da-af7b-1435c769b35f'),
 UUID('eebc547e-bd11-46db-9a17-5d5f224bbeac'),
 UUID('5461eb58-a836-48b2-a691-c7773a9a1d01'),
 UUID('4c48de2e-a3e2-4aa2-92d6-3091f6cfa66f'),
 UUID('17960fc9-58c5-474b-849b-1d3343ae0878'),
 UUID('37ceac17-2f2c-494f-b442-f0469375acce'),
 UUID('6363813d-e1dc-4ac2-9062-da0e87bcfb6d'),
 UUID('a68c2ac0-fa4c-4182-8af2-a7435098e658'),
 UUID('57243f1d-1c6a-4a19-adc4-97e57e588389'),
 UUID('aa7b5cdc-ea58-4354-b8de-60dbe6cc19c3'),
 UUID('ec9261fc-402b-4f49-9e8c-89443084c85c'),
 UUID('4b533ec7-02d4-4e52-a56d-90f15018e685'),
 UUID('0c9e9bee-0b26-4df6-9160-8fd53959ec26'),
 UUID('37685ac6-ed01-4682-a484-6c19f380ea98'),
 UUID('cbf5ee

In [20]:
r.info('memory')['used_memory'] - used_memory_before

2175672

Потратили в 2 раза меньше памяти для сохранения рекомендаций 🎉🎉